## Multiple EM for Motif Elicitation

In [1]:
import numpy as np
from Bio import SeqIO
from sklearn.preprocessing import normalize

In [15]:
w = 8 #width
l = 101 #length

# Helper dict for indexing
let_dict = {"A":0, "C":1, "G":2, "T":3}

## 1. Read in Fasta Sequences

In [16]:
fasta = SeqIO.parse("motif-regions.fa", "fasta")

In [17]:
seqs = []
for record in SeqIO.parse("motif-regions.fa", "fasta"):
    seqs.append(str(record.seq))
print(seqs)


['CGCTCATAAAACCATGCGCGTGTTTTTCTTCCACGGCGTTCTAGAAAAAAAAGAAAAATTCAAAAAGTTCTGCGAAACTCGAAAGAACCAGAATTGTTCGA', 'TCCACACTGACGGGGTCACGTGTGTTTATTCTGTTTTCTTCTAACGGCTTCTGTAGATTTCCGGACGGTTCCACGCAGCGCCACAGCTTGGTTCCAGATCG', 'CGTTCATGAAAGAGGCAACGATGGTGATCGATTATTTCCAGGAATTGTTCCATGACGTTCTACGTGGTTCTCTTGGTCGGGTAATACAATCAGGGTAACAT', 'GTAAAGTTACTGACACTTTTTTTTCTAGAAAGTTCCGGAAAATTGCGACACTCGGTGGAGCTCGAGAGTTGTATCCAGTTTTCTTGTTCGGCGATATTCCG', 'TGTCAGCGCGAGATGCCTTCCTCTTTTCTAACCTCCAAAAAACTTCCAGAAGATTCCAGTCCAAGAGGTATATCATTTCAAACGAATAAGCTGATTACAAC', 'GAGAAAAAACGCAAATGACAGCTTCTAAACGTTCCGTGTGCTTTCTTTCTAGAATGTTCTGGAAAGTTTACAACAATCCACAAGAACGAAAATGCCGTTGA', 'ATTACCCGCCTTTCTGTTTTCTGGGCACTTTTCTTTCTAGAAGGTGAAAGAACAATTTTTCTCGTTTTCTCGAACTTCCACCAAGCGTTGGGTAATGAGGG', 'GAAATTTCGTCAAAACCTCGAAGCTTTTTTTTTCGAAATCTTCGATGGAATGTTCTAGAAAAAAACTTCGAGAAAGTGAATCGGAAGAACTTCTCGGAAAA', 'AAAGCCCTTACTAACCCTACAATAAATTGTGCCGAAACCCTCTGGAGTTTTCTAGAATATTCTAGCCCCATCAGGGCTAGAATATCCTAAAAGTTTATAGT', 'GATGCTCTATTGCCTTTTCATTTCTTGAAAAAGAAATGAAAAAATTTTCTATT

## 2. Create p_0

In [18]:
def init_p(l, w, seqs, let):
    p = np.zeros((4, w+1))

    # set a uniform background for each A C G T

    for i in range(len(p)):
        

        p[i][0] = 0.25

    # set motif positions
    for i in range(l-w+1):
        for sequence in seqs:
            for j in range(w):

                n = sequence[i+j]
                p[let[n]][j+1] +=1
                # Fill in p_0
                
    # normalize columns to sum to 1
    p = normalize(p, axis = 0, norm = 'l1')
    
    return p

## 3. Fill in EM iteration

In [19]:
def run_EM(l,w, seqs, let, init_p, up_prob, up_motif, epsilon = 2**-64):

    no_change = False
    
    # set an initial p_t_1
    p_t_1 = init_p(l, w, seqs, let)
        
    while not no_change:
        # E step        
        z_t = up_motif(l, w, p_t_1, seqs, let)
        
        # M step 
                
        p_t = up_prob(w, z_t, seqs, let)
        
        diff = np.subtract(p_t, p_t_1)
        
        # Write a condition to stop the EM iterations (use epsilon and diff)
        if diff.any() <= epsilon:        
            no_change = True
        else:
            # Update p_t_1
            p_t_1 = p_t
                
    return p_t, z_t

## 4. Fill in function to update z_t

In [7]:
def up_motif(l, w, p_t_1, seqs, let):
    z_t = np.zeros((len(seqs), l-w+1))

    for i, sequence in enumerate(seqs):
        for j in range(l-w+1):
            
            # Fill in z_t using p_t_1
            # Ignore background as we're assuming 0.25 for all 4
            z_t[i][j] = 1
            
            for s in range(len(sequence)):
                n = sequence[s]
                if s >= j and s < j+w: 

                    z_t[i][j] *= p_t_1[let[n]][s-j+1]
                else:
                    z_t[i][j] *= p_t_1[let[n]][0] #background
            
    
    # Normalize z_t
   
    z_t = normalize(z_t, axis = 1, norm = 'l1') #axis = 0?
        
    return z_t

## 5. Fill in function to update p_t

In [8]:
def up_prob(w, z_t, seqs, let):
    p_t = np.zeros((4, w+1)) 
    n = np.zeros((4, w+1))
    
    # Fill in n for k > 0 - motif
    for s in range(1, w+1):
        for letter in let.keys():
            sum_z = 0
            for i, sequence in enumerate(seqs):
                
                # Write j_vals according to the condition seen in lecture
               
                j_vals = [a-s+1 for a in range(len(sequence)-w+1) if sequence[a] == letter] 
                                
                # Add to the sum using z_t
                
                for j in j_vals:
                    sum_z += z_t[i][j]
            
            # Fill in the correct indices
            
            n[let[letter]][s] = sum_z
                 
    # Fill in n for k == 0
    
    # May help to make the next step easier
    joined_seq = "".join(seqs)
    
    # Create a dict with total counts of A,C,G,T

    counts = {}
    
    for nuc in joined_seq: 
        if nuc in counts:
            counts[nuc] += 1
        else:
            counts[nuc] = 1
        
    # Sum across the rows of n

    sum_n_j = np.sum(n,axis = 0)    
        
    for letter in let.keys():
        
        # Fill in the correct indices and its value
        n[let[letter]][0] = counts[letter] - sum_n_j[let[letter]]
    
    p_t = (n+1)/(sum_n_j+4)
        
    return p_t

## 6. Run the EM to find the final p and z

In [9]:
p_end, z_end = run_EM(l, w, seqs, let_dict, init_p, up_prob, up_motif)

## 7. Determine Motifs

In [11]:
motif_indices = np.argmax(z_end, axis=1)
print(motif_indices)

# Get the 'width' characters long motifs using seqs
for index in motif_indices:
    motifs = [sequence[index:index+8] for sequence in seqs]
print(motifs)

[51 62 40 36 47 60 48 46 53 38 53 34 55 38 55 42 47 54 21 52 46 43 52 45
 66 24 34 13 21 44 21 84 50  6  9 64 20 82 43 50 48 61 74 93 79 30 33 78
 30 40]
['CTAGAAAA', 'CTAACGGC', 'GGAATTGT', 'AATTGCGA', 'AACTTCCA', 'CTTTCTTT', 'AAGGTGAA', 'TTCGATGG', 'TCTGGAGT', 'AAAATTTT', 'CCCAGTTC', 'TTCTAGAA', 'CGCTCGAT', 'AACGTTCT', 'GCGCTTAA', 'CTAGAACA', 'AGGTGTAA', 'TTCTAGAA', 'TAACTATA', 'GTGTAGTA', 'GCCCATGG', 'AACGGAAC', 'TTCTATTT', 'GCAATCGA', 'ACGCGAGA', 'AAATTTGA', 'ATTTGGGT', 'TTACTTTT', 'GACGCATC', 'TTCTAGAA', 'TCATAGAA', 'GTTCGTAG', 'TTGGTTTT', 'TTCTTAAC', 'GAAGACAG', 'CGTGGAAG', 'AAGAAGAA', 'TTTTCCAT', 'TCTAGAAC', 'GAAACGCA', 'AAAAAAAA', 'AGGGGCTA', 'TCTATTTC', 'TTTTCCAA', 'TCTGGAAA', 'GAAACAGG', 'CTCTGTGT', 'CAATAAAA', 'AAAGAAAA', 'AGATTGTT']
